In [1]:
%cd '/content/drive/MyDrive/UCO/projects/epilepsy'

/content/drive/MyDrive/UCO/projects/epilepsy


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
52 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [3]:
import pandas as pd

from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *

In [4]:
cohort_new = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort2024')

In [5]:
cohort_como = spark.read.parquet('/content/drive/MyDrive/UCO/projects/epilepsy/allcohort_new_como')

In [6]:
cohort_como_reformat= cohort_como.withColumn('comorbidityid',expr("regexp_replace(comorbidityid, '([0-9a-zA-Z]+\\.[0-9a-zA-Z]).*', '$1')").cast("string"))

In [7]:
def get_top_comorbidity(spark, como, topn):
    top_como = como.groupBy('comorbidityid').count().orderBy(col('count').desc())\
        .select(collect_list('comorbidityid')).collect()[0]\
            .__getitem__('collect_list(comorbidityid)')[0:topn]

#     top_como_coded = ["n"+x.replace('.','_') for x in top_como]
#     out = {top_como[i]:top_como_coded[i] for i in range(len(top_como))} # store in a dictionary
    print(top_como)
    return top_como

In [8]:
cohort_como_length = cohort_como_reformat.join(cohort_new.select('personid','date'),'personid').withColumn('len_como',months_between(col('date'),col('effectivedate'))/12)

In [9]:
# prompt: make the len_como integer

cohort_como_length = cohort_como_length.withColumn("len_como", col("len_como").cast("integer"))


In [10]:
# prompt: generate a function to calculate the percentage of patients with 50% or more top comorbidities

def calculate_percentage_patients_with_top_comorbidities(spark, cohort_como_length, topn):
    """
    Calculates the percentage of patients with a specified number or more top comorbidities.

    Args:
        spark: SparkSession object.
        cohort_como_length_filter_selected: DataFrame containing patient and comorbidity data.
        threshold: The minimum number of distinct comorbidities a patient must have to be included in the calculation.

    Returns:
        float: The percentage of patients with the specified threshold or more distinct comorbidities.
    """
    top_comorbidity = get_top_comorbidity(spark, cohort_como_length, topn)
    cohort_como_length_filter = cohort_como_length.filter(col('comorbidityid').isin(top_comorbidity))
    cohort_como_length = cohort_como_length.withColumn("len_como", col("len_como").cast("integer"))
    cohort_como_length_filter_selected = cohort_como_length_filter.select('personid', 'comorbidityid', 'len_como').dropDuplicates()
    threshold = 0.5 * topn
    print(f"Threshold: {threshold}")
    distinct_persons = cohort_como_length_filter_selected.groupBy('personid').agg(countDistinct('comorbidityid').alias('distinct_comorbidity_count'))
    result = distinct_persons.filter(f"distinct_comorbidity_count >= {threshold}").count()
    total_patients = distinct_persons.count()
    percentage = (result / total_patients) * 100 if total_patients > 0 else 0
    print(f"Percentage of patients with 50% more distinct comorbidityid: {percentage:.4f}%")
    return percentage

In [16]:
percentage = calculate_percentage_patients_with_top_comorbidities(spark, cohort_como_length, 20)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6']
Threshold: 10.0
Percentage of patients with 50% more distinct comorbidityid: 2.16%


In [17]:
percentage = calculate_percentage_patients_with_top_comorbidities(spark, cohort_como_length, 50)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1']
Threshold: 25.0
Percentage of patients with 50% more distinct comorbidityid: 0.23%


In [18]:
percentage = calculate_percentage_patients_with_top_comorbidities(spark, cohort_como_length, 100)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1', 'H66.9', 'Z20.8', 'G47.3', 'J44.9', 'E78.2', 'Y92.4', 'R19.7', 'Z51.8', '493.9', 'R07.8', 'M54.9', '780.6', 'Z72.0', '719.4', 'E55.9', 'R41.8', '786.5', 'F10.1', 'S16.1', 'I48.0', 'Z98.8', 'S00.0', 'E66.9', 'T14.8', 'F03.9', '787.0', 'N18.3', 'E11.6', 'Z68.5', '530.8', 'M19.9', '427.3', 'Z13.8', 'W01.0', 'Z68.2', 'R09.8', '462', 'Z68.3', 'Z12.3', 'R11.2', 'M81.0', '414.0', 'E87.6', '382.9', 'R53.1', '244.9', 'Z01.8', 'E86.0', 'I50.9', 'W18.3']
Threshold: 50.0
Percentage of patients with 50% more distinct comorbidityid: 0.03%


In [11]:
percentage = calculate_percentage_patients_with_top_comorbidities(spark, cohort_como_length, 200)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1', 'H66.9', 'Z20.8', 'G47.3', 'J44.9', 'E78.2', 'Y92.4', 'R19.7', 'Z51.8', '493.9', 'R07.8', 'M54.9', '780.6', 'Z72.0', '719.4', 'E55.9', 'R41.8', '786.5', 'F10.1', 'S16.1', 'I48.0', 'Z98.8', 'S00.0', 'E66.9', 'T14.8', 'F03.9', '787.0', 'N18.3', 'E11.6', 'Z68.5', '530.8', 'M19.9', '427.3', 'Z13.8', 'W01.0', 'Z68.2', 'R09.8', '462', 'Z68.3', 'Z12.3', 'R11.2', 'M81.0', '414.0', 'E87.6', '382.9', 'R53.1', '244.9', 'Z01.8', 'E86.0', 'I50.9', 'W18.3', '786.2', 'R11.0', 'J30.9', 'F41.1', 'R73.0', 'X58.X', 'N17.9', '300.0', 'M62.8', 'V89.2', 'J18.9', 'R10.1', 'B34.

In [10]:
# prompt: calculate percentage of comorbidites that occur more than once for patients

def calculate_percentage_patients_with_multiple_comorbidities(spark, cohort_como_length, topn):
    """
    Calculates the percentage of patients with comorbidities occurring more than once.

    Args:
        spark: SparkSession object.
        cohort_como_length: DataFrame containing patient and comorbidity data.

    Returns:
        float: The percentage of patients with comorbidities occurring more than once.
    """
    top_comorbidity = get_top_comorbidity(spark, cohort_como_length, topn)
    cohort_como_length_filter = cohort_como_length.filter(col('comorbidityid').isin(top_comorbidity))
    cohort_como_length = cohort_como_length.withColumn("len_como", col("len_como").cast("integer"))
    cohort_como_length_filter_selected = cohort_como_length_filter.select('personid', 'comorbidityid', 'len_como').dropDuplicates()
    comorbidity_counts = cohort_como_length_filter_selected.groupBy("personid", "comorbidityid").count()
    patients_with_multiple_comorbidities = comorbidity_counts.filter(col("count") > 1)

    total_patients = cohort_como_length.select("personid").distinct().count()
    patients_with_multiple_comorbidities_count = patients_with_multiple_comorbidities.select("personid").distinct().count()

    percentage = (patients_with_multiple_comorbidities_count / total_patients) * 100 if total_patients > 0 else 0

    print(f"Percentage of patients with comorbidities occurring more than once: {percentage:.4f}%")
    return percentage



In [11]:
percentage = calculate_percentage_patients_with_multiple_comorbidities(spark, cohort_como_length, 20)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6']
Percentage of patients with comorbidities occurring more than once: 19.7581%


In [12]:
percentage = calculate_percentage_patients_with_multiple_comorbidities(spark, cohort_como_length, 50)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1']
Percentage of patients with comorbidities occurring more than once: 25.1898%


In [12]:
percentage = calculate_percentage_patients_with_multiple_comorbidities(spark, cohort_como_length, 100)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1', 'H66.9', 'Z20.8', 'G47.3', 'J44.9', 'E78.2', 'Y92.4', 'R19.7', 'Z51.8', '493.9', 'R07.8', 'M54.9', '780.6', 'Z72.0', '719.4', 'E55.9', 'R41.8', '786.5', 'F10.1', 'S16.1', 'I48.0', 'Z98.8', 'S00.0', 'E66.9', 'T14.8', 'F03.9', '787.0', 'N18.3', 'E11.6', 'Z68.5', '530.8', 'M19.9', '427.3', 'Z13.8', 'W01.0', 'Z68.2', 'R09.8', '462', 'Z68.3', 'Z12.3', 'R11.2', 'M81.0', '414.0', 'E87.6', '382.9', 'R53.1', '244.9', 'Z01.8', 'E86.0', 'I50.9', 'W18.3']
Percentage of patients with comorbidities occurring more than once: 27.6334%


In [11]:
percentage = calculate_percentage_patients_with_multiple_comorbidities(spark, cohort_como_length, 200)

['I10', 'Z79.8', 'Z00.1', 'Z23', 'M25.5', 'E78.5', 'E11.9', 'W19.X', 'J06.9', 'I25.1', 'E03.9', 'K21.9', 'Y92.0', 'Z79.0', 'F41.9', '401.9', 'R51', 'R05', 'I48.9', 'M79.6', 'Z00.0', 'F32.9', 'R50.9', 'J45.9', 'R10.9', 'V20.2', 'Z87.8', 'N39.0', 'V58.6', 'J02.9', 'S00.8', 'R42', 'E78.0', 'F17.2', 'Z86.7', 'D64.9', '272.4', 'R55', 'R06.0', '789.0', '250.0', 'M54.2', 'M54.5', 'S01.8', 'R07.9', 'K59.0', '465.9', 'R53.8', 'S01.0', 'R11.1', 'H66.9', 'Z20.8', 'G47.3', 'J44.9', 'E78.2', 'Y92.4', 'R19.7', 'Z51.8', '493.9', 'R07.8', 'M54.9', '780.6', 'Z72.0', '719.4', 'E55.9', 'R41.8', '786.5', 'F10.1', 'S16.1', 'I48.0', 'Z98.8', 'S00.0', 'E66.9', 'T14.8', 'F03.9', '787.0', 'N18.3', 'E11.6', 'Z68.5', '530.8', 'M19.9', '427.3', 'Z13.8', 'W01.0', 'Z68.2', 'R09.8', '462', 'Z68.3', 'Z12.3', 'R11.2', 'M81.0', '414.0', 'E87.6', '382.9', 'R53.1', '244.9', 'Z01.8', 'E86.0', 'I50.9', 'W18.3', '786.2', 'R11.0', 'J30.9', 'F41.1', 'R73.0', 'X58.X', 'N17.9', '300.0', 'M62.8', 'V89.2', 'J18.9', 'R10.1', 'B34.